# LR & DA

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from main import CIFAR10_dataset, CIFAR10_trainer
import torch
import torchvision
import torch.nn as nn
from tqdm import tqdm
import multiprocessing
import torch.optim as optim
import torch.nn.functional as  F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms

In [3]:
da_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=5, translate=(0.2, 0.2)),
    transforms.ToTensor(),
])

train_dataset = CIFAR10_dataset(partition="train", transform=da_train)
test_dataset = CIFAR10_dataset(partition="test")

####################################################################
# DataLoader Class
####################################################################

batch_size = 100
num_workers = multiprocessing.cpu_count()-1
print("Num workers", num_workers)
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=num_workers)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=num_workers)


Loading CIFAR10  train  Dataset...
Files already downloaded and verified
	Total Len.:  50000 
 --------------------------------------------------

Loading CIFAR10  test  Dataset...
Files already downloaded and verified
	Total Len.:  10000 
 --------------------------------------------------
Num workers 11


In [4]:


####################################################################
# Neural Network Class
####################################################################

# Define the CNN model
# Define ConvBlocks to create a CNN
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
    def forward(self, x):
        
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.pool(x)

        return x

# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        
        self.conv_layers = nn.Sequential(
                ConvBlock(3, 32),
                ConvBlock(32, 64),
                ConvBlock(64, 128),
                ConvBlock(128, 256),
                ConvBlock(256, 512)
            )
        
        self.fc1 = nn.Linear(512, 512)
        self.fc2 = nn.Linear(512, num_classes)

        self.relu = nn.ReLU()

    def forward(self, x):
        for layer in self.conv_layers:
            x = layer(x)

        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x


In [5]:
# Instantiating the network and printing its architecture
num_classes = 10
net = SimpleCNN(num_classes)
print(net)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Params: ", count_parameters(net))

####################################################################
# Training settings
####################################################################

# Training hyperparameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, weight_decay=1e-6, momentum=0.9)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=10, min_lr=0.00001)
epochs = 100

SimpleCNN(
  (conv_layers): Sequential(
    (0): ConvBlock(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): ConvBlock(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): ConvBlock(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (3): ConvBlock(
      (conv): C

In [6]:
trainer = CIFAR10_trainer(net, train_dataloader, test_dataloader, optimizer,criterion, epochs, lr_scheduler)

In [7]:
trainer.train()


---- Start Training ----


Test 0: 100%|██████████| 100/100 [00:01<00:00, 66.22batch/s]

[Epoch 1] Train Loss: 1.475960 - Test Loss: 1.542921 - Train Error: 54.63% - Test Error: 52.50%



Test 1: 100%|██████████| 100/100 [00:01<00:00, 66.39batch/s]

[Epoch 2] Train Loss: 1.117050 - Test Loss: 1.167565 - Train Error: 40.29% - Test Error: 42.33%


...

[Epoch 73] Train Loss: 0.211778 - Test Loss: 0.399338 - Train Error: 7.48% - Test Error: 12.51%



Test 73: 100%|██████████| 100/100 [00:01<00:00, 64.96batch/s]

[Epoch 74] Train Loss: 0.214185 - Test Loss: 0.397276 - Train Error: 7.64% - Test Error: 12.46%



Test 74: 100%|██████████| 100/100 [00:01<00:00, 63.95batch/s]

[Epoch 75] Train Loss: 0.216648 - Test Loss: 0.398185 - Train Error: 7.71% - Test Error: 12.53%

Early Stopping at epoch  74

BEST TEST ERROR:  12.38  in epoch  58
